### Import necessary libraries

In [1]:
!pip install --upgrade scikit-learn==1.6.1 imbalanced-learn category-encoders numpy==1.26.4 pandas scipy

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import seaborn as sns
import os

2025-07-18 16:31:07.071342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752856267.256616      87 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752856267.311615      87 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Loading Data

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel2.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/SampleSubmission.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel1.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Test.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shx
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.dbf
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shp
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.dbf
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shx
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shp


In [4]:
# Load Sentinel-1 and 2
s1 = pd.read_csv("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel1.csv").drop(columns=['date'])
s2 = pd.read_csv("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel2.csv").drop(columns=['date'])

In [5]:
s1.head()

,ID,VH,VV,orbit,polarization,rel_orbit,translated_lat,translated_lon
0,ID_AFQOFP,-21.479683,-16.633259,DESCENDING,"[VV, VH]",78.0,41.652292,72.144256
1,ID_AFQOFP,-24.769110,-15.943674,DESCENDING,"[VV, VH]",78.0,41.652289,72.144375
2,ID_AFQOFP,-25.370838,-15.185609,DESCENDING,"[VV, VH]",78.0,41.652286,72.144495
3,ID_AFQOFP,-24.134005,-16.351102,DESCENDING,"[VV, VH]",78.0,41.652283,72.144614
4,ID_AFQOFP,-20.654249,-16.792723,DESCENDING,"[VV, VH]",78.0,41.652280,72.144733


In [6]:
s2.head()

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,ID,cloud_pct,solar_azimuth,solar_zenith,translated_lat,translated_lon
0,2169,1820,1328,1610,1670,1985,2446,2628,2598,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935173,71.617062
1,2151,1770,1306,1586,1640,1961,2495,2691,2684,2732,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935171,71.617180
2,2169,1820,1456,1674,1808,1985,2446,2628,2486,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935085,71.616940
3,2169,1820,1284,1604,1658,1985,2446,2628,2658,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935083,71.617059
4,2151,1770,1242,1522,1564,1961,2495,2691,2696,2732,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935081,71.617177


### Data Preparation

In [7]:
# shape of sentinel 1 & 2
print('s1 shape:', s1.shape)
print('\n')
print('s2 shape:', s2.shape)

s1 shape: (1752570, 8)


s2 shape: (5610393, 16)


In [8]:
# Sentinel 1 & 2 information
s1.info()
print('-'*50)
s2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752570 entries, 0 to 1752569
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              object 
 1   VH              float64
 2   VV              float64
 3   orbit           object 
 4   polarization    object 
 5   rel_orbit       float64
 6   translated_lat  float64
 7   translated_lon  float64
dtypes: float64(5), object(3)
memory usage: 107.0+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5610393 entries, 0 to 5610392
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   B11             int64  
 1   B12             int64  
 2   B2              int64  
 3   B3              int64  
 4   B4              int64  
 5   B5              int64  
 6   B6              int64  
 7   B7              int64  
 8   B8              int64  
 9   B8A             int64  
 10  ID              object 
 11

In [9]:
# Check null values
print('s1')
print(s1.isna().sum())
print('-'*50)
print('s2')
print(s2.isna().sum())

s1
ID                0
VH                0
VV                0
orbit             0
polarization      0
rel_orbit         0
translated_lat    0
translated_lon    0
dtype: int64
--------------------------------------------------
s2
B11               0
B12               0
B2                0
B3                0
B4                0
B5                0
B6                0
B7                0
B8                0
B8A               0
ID                0
cloud_pct         0
solar_azimuth     0
solar_zenith      0
translated_lat    0
translated_lon    0
dtype: int64


### Extract Labelled GeoData

In [10]:
def load_training_data():
    fergana = gpd.read_file("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shp")
    orenburg = gpd.read_file("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shp")
    
    # Manually set CRS
    fergana.crs = "EPSG:32642"
    orenburg.crs = "EPSG:32640"
    
    # Convert to WGS84
    fergana = fergana.to_crs("EPSG:4326")
    orenburg = orenburg.to_crs("EPSG:4326")
    
    # Convert to centroids
    fergana['geometry'] = fergana.geometry.centroid
    orenburg['geometry'] = orenburg.geometry.centroid
    
    # Combine datasets
    gdf = gpd.GeoDataFrame(pd.concat([fergana, orenburg]), crs="EPSG:4326")
    gdf['lon'] = gdf.geometry.x
    gdf['lat'] = gdf.geometry.y
    
    return gdf[['Cropland', 'lon', 'lat']]

### Feature Engineering

In [11]:
# Robust feature aggregation
def aggregate_features(df, id_col='ID'):
    """Aggregate time-series data using mean and std for numeric columns only"""
    # Convert to numeric and handle errors
    numeric_df = df.copy()
    for col in numeric_df.columns:
        if col != id_col:
            numeric_df[col] = pd.to_numeric(numeric_df[col], errors='coerce')
    
    # Fill any remaining NaNs
    numeric_df = numeric_df.fillna(0)
    
    # Select only numeric columns
    numeric_cols = numeric_df.select_dtypes(include=[np.number]).columns.tolist()
    if id_col not in numeric_cols:
        numeric_cols.append(id_col)
    
    # Group and aggregate
    agg_df = numeric_df[numeric_cols].groupby(id_col).agg(['mean', 'std'])
    
    # Flatten column names
    agg_df.columns = [f'{col[0]}_{col[1]}' for col in agg_df.columns]
    return agg_df.reset_index()

In [12]:
# Feature engineering
def calculate_vegetation_indices(df):
    """Calculate vegetation indices"""
    # Convert to numeric
    for band in ['B2', 'B3', 'B4', 'B8', 'B11']:
        df[band] = pd.to_numeric(df[band], errors='coerce')
    
    df['NDVI'] = (df['B8'] - df['B4']) / (df['B8'] + df['B4'] + 1e-8)
    df['NDWI'] = (df['B3'] - df['B8']) / (df['B3'] + df['B8'] + 1e-8)
    return df

In [13]:
# Load training data
train_gdf = load_training_data()
print(f"Loaded {len(train_gdf)} training samples")

Loaded 1000 training samples


/tmp/ipykernel_87/1212045649.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fergana['geometry'] = fergana.geometry.centroid
/tmp/ipykernel_87/1212045649.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  orenburg['geometry'] = orenburg.geometry.centroid


In [14]:
# Process Sentinel-2 data
s2 = calculate_vegetation_indices(s2)
print("Sentinel-2 columns:", s2.columns.tolist())
s2_agg = aggregate_features(s2)

Sentinel-2 columns: ['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'ID', 'cloud_pct', 'solar_azimuth', 'solar_zenith', 'translated_lat', 'translated_lon', 'NDVI', 'NDWI']


In [15]:
# Process Sentinel-1 data
print("\nSentinel-1 columns:", s1.columns.tolist())


Sentinel-1 columns: ['ID', 'VH', 'VV', 'orbit', 'polarization', 'rel_orbit', 'translated_lat', 'translated_lon']


In [16]:
# Convert Sentinel-1 to numeric
for col in ['VV', 'VH']:
    s1[col] = pd.to_numeric(s1[col], errors='coerce')
s1 = s1.fillna(0)

In [17]:
s1_agg = aggregate_features(s1)
print("\nSentinel-1 aggregated columns:", s1_agg.columns.tolist())


Sentinel-1 aggregated columns: ['ID', 'VH_mean', 'VH_std', 'VV_mean', 'VV_std', 'orbit_mean', 'orbit_std', 'polarization_mean', 'polarization_std', 'rel_orbit_mean', 'rel_orbit_std', 'translated_lat_mean', 'translated_lat_std', 'translated_lon_mean', 'translated_lon_std']


#### Merge satellite data

In [18]:
# Merge satellite data
satellite_data = pd.merge(s2_agg, s1_agg, on='ID', how='inner')
print(f"\nMerged satellite data shape: {satellite_data.shape}")


Merged satellite data shape: (600, 49)


In [19]:
# Get mean coordinates
coords = s2.groupby('ID')[['translated_lon', 'translated_lat']].mean().reset_index()

In [24]:
# Check if satellite coordinates are in UTM range (large numbers)
if coords['translated_lon'].max() > 180 or coords['translated_lat'].max() > 90:
    print("Converting satellite coordinates to WGS84...")
    
    # Create UTM geometry (assuming zone 40 for Fergana, 41 for Orenburg)
    from pyproj import Transformer
    
    # Transform UTM to WGS84
    transformer = Transformer.from_crs("EPSG:32640", "EPSG:4326", always_xy=True)  # Orenburg
    # For Fergana use EPSG:32642
    
    # Apply transformation
    wgs84_coords = []
    for _, row in coords.iterrows():
        # Check which zone it belongs to based on position
        if row['translated_lon'] > 500000:  # UTM Easting
            # Orenburg zone (EPSG:32640)
            lon, lat = transformer.transform(row['translated_lon'], row['translated_lat'])
        else:
            # Fergana zone (EPSG:32642)
            lon, lat = transformer.transform(row['translated_lon'], row['translated_lat'])
        wgs84_coords.append([lon, lat])
    
    coords[['lon_wgs84', 'lat_wgs84']] = np.array(wgs84_coords)
    # Use converted coordinates for matching
    coords['translated_lon'] = coords['lon_wgs84']
    coords['translated_lat'] = coords['lat_wgs84']

In [25]:
# Match training labels to satellite points
tree = KDTree(train_gdf[['lon', 'lat']].values)
distances, indices = tree.query(coords[['translated_lon', 'translated_lat']].values, k=1)

In [26]:
# Set distance threshold
MAX_DIST = 0.1
valid_mask = distances.flatten() <= MAX_DIST

In [27]:
# Create labels
labels = pd.DataFrame({
    'ID': coords['ID'],
    'label': np.where(valid_mask, train_gdf.iloc[indices.flatten()]['Cropland'].values, -1),
    'distance': distances.flatten()
})

In [28]:
# Filter valid labels
valid_labels = labels[labels['label'] != -1].drop(columns='distance')
print(f"Matched {len(valid_labels)}/{len(coords)} points to training labels")

# Merge features with labels
full_data = pd.merge(satellite_data, valid_labels, on='ID')
print(f"Final dataset size: {full_data.shape}")

Matched 0/600 points to training labels
Final dataset size: (0, 50)


In [ ]:
train_df.head()

In [ ]:
# Train-test split
X = train_df.drop(columns=["ID", "label"])
y = train_df["label"]
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

### CNN Model

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Prepare data for CNN: reshape to (samples, features, 1)
X_train_cnn = np.expand_dims(X_train_scaled, axis=2)
X_val_cnn = np.expand_dims(X_val_scaled, axis=2)

# Encode labels if not already 0/1
num_classes = len(np.unique(y_train))
y_train_cnn = to_categorical(y_train, num_classes)
y_val_cnn = to_categorical(y_val, num_classes)

# Build a simple 1D CNN model with Input layer
model = Sequential([
    Input(shape=(X_train_cnn.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.2),
    Conv1D(64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_cnn_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model with the callback
model.fit(
    X_train_cnn, y_train_cnn,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_cnn, y_val_cnn),
    callbacks=[checkpoint],
    verbose=2
)

# Load the best weights before evaluation
model.load_weights('best_cnn_model.h5')

In [ ]:
# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val_cnn, y_val_cnn, verbose=0)
print("Validation Accuracy:", val_acc)

In [ ]:
# Prepare test set
test_meta = pd.read_csv("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Test.csv")
test_ids = test_meta["ID"].unique()
s1_test = s1[s1["ID"].isin(test_ids)]
s2_test = s2[s2["ID"].isin(test_ids)]

In [ ]:
# Drop date + aggregate test features
s1_test_feats = aggregate_features(s1_test[["ID", "VH", "VV"]])
s2_test_feats = aggregate_features(s2_test[["ID", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"]])
s1_test_feats.columns = ['_'.join(col).strip("_") for col in s1_test_feats.columns.values]
s2_test_feats.columns = ['_'.join(col).strip("_") for col in s2_test_feats.columns.values]

In [ ]:
# Merge test features
test_df = s2_test_feats.merge(s1_test_feats, on="ID", how="outer").fillna(0)
X_test = test_df.drop(columns=["ID"])
test_preds = model.predict(X_test)

In [ ]:
# Save predictions to CSV
predicted_labels = np.argmax(test_preds, axis=1)
submission = pd.DataFrame({'ID': test_df['ID'], 'label': predicted_labels})
submission.to_csv('submission.csv', index=False)